In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/zip-ragbot-new/ragbot_databse_new/ragbot_databse_new/qdrant_db_backup/meta.json
/kaggle/input/zip-ragbot-new/ragbot_databse_new/ragbot_databse_new/qdrant_db_backup/collection/your_collection/storage.sqlite
/kaggle/input/zip-ragbot-new/ragbot_databse_new/ragbot_databse_new/lora_adapter_latest/adapter_model.safetensors
/kaggle/input/zip-ragbot-new/ragbot_databse_new/ragbot_databse_new/lora_adapter_latest/adapter_config.json
/kaggle/input/zip-ragbot-new/ragbot_databse_new/ragbot_databse_new/lora_adapter_latest/README.md
/kaggle/input/zip-ragbot-new/ragbot_databse_new/ragbot_databse_new/lora_adapter_latest/tokenizer.json
/kaggle/input/zip-ragbot-new/ragbot_databse_new/ragbot_databse_new/lora_adapter_latest/tokenizer_config.json
/kaggle/input/zip-ragbot-new/ragbot_databse_new/ragbot_databse_new/lora_adapter_latest/chat_template.jinja
/kaggle/input/zip-ragbot-new/ragbot_databse_new/ragbot_databse_new/lora_adapter_latest/special_tokens_map.json


In [2]:
!pip install qdrant-client
!pip install peft transformers accelerate bitsandbytes --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 6.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 23.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━

In [3]:
! pip install -U bitsandbytes

In [4]:
import bitsandbytes as bnb
print(bnb.__version__)


0.46.1


In [5]:
!pip install -U peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 9.3 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: peft
    Found existing installation: peft 0.15.2
    Uninstalling peft-0.15.2:
      Successfully uninstalled peft-0.15.2


In [6]:
import os
import torch
# from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import torch
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, Distance, VectorParams
import uuid
import shutil

2025-08-05 15:32:15.378950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754407935.739793      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754407935.844777      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("HF_TOKEN")
from huggingface_hub import login
login(token=token)

In [8]:
import zipfile
import shutil

# Copy zip to working (if needed)
shutil.copytree("/kaggle/input/zip-ragbot-new/ragbot_databse_new/ragbot_databse_new/", "/kaggle/working/ragbot_databse_new/")

# # Unzip in working directory
# with zipfile.ZipFile("/kaggle/working/archive.zip", 'r') as zip_ref:
#     zip_ref.extractall("/kaggle/working/your_folder")


'/kaggle/working/ragbot_databse_new/'

In [9]:
# # Qdrant connection
# qdrant = QdrantClient(path="vector_store_api/qdrant_db")

# collection_name = "my_collection"

# # Ensure collection exists (matching your folder)
# if collection_name not in [c.name for c in qdrant.get_collections().collections]:
#     qdrant.create_collection(
#         collection_name=collection_name,
#         vectors_config=models.VectorParams(
#             size=embedding_dim,
#             distance=models.Distance.COSINE
#         )
#     )

# Initialize Qdrant
qdrant = QdrantClient(path="/kaggle/working/ragbot_databse_new/qdrant_db_backup")  # Or path/to/db if persistent
collection_name = "your_collection"

# Connect to existing Qdrant vector database
# qdrant = QdrantClient(path="/content/drive/MyDrive/vector_store_api/qdrant_db")
# collection_name = "my_collection"  # collection name from pipeline

# Check if collection exists
# qdrant = QdrantClient(path=qdrant_path)
collections = [c.name for c in qdrant.get_collections().collections]
if collection_name not in collections:
    raise ValueError(f"Collection '{collection_name}' not found. Make sure the DB is loaded and ingested with medical facts only!")

# Load the embedding model (same one used during indexing)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # or the one you used




/tmp/ipykernel_36/3077486002.py:17: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Collection <your_collection> contains 87540 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  qdrant = QdrantClient(path="/kaggle/working/ragbot_databse_new/qdrant_db_backup")  # Or path/to/db if persistent


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# # (OPTIONAL) Clean your vector DB if polluted with "Prompt: ..." records
# # This cell will delete all points containing 'Prompt:' in the text field.
# # Uncomment and run ONCE if your DB is contaminated.

# '''
# all_points = []
# offset = None
# while True:
#     points, offset = qdrant.scroll(collection_name=collection_name, limit=100, with_payload=True, offset=offset)
#     all_points.extend(points)
#     if offset is None:
#         break

# ids_to_delete = [
#     point.id for point in all_points
#     if any("prompt:" in str(point.payload.get(k, "")).lower() for k in ["text", "chunk", "chunk_text", "content"])
# ]
# if ids_to_delete:
#     qdrant.delete(collection_name=collection_name, points_selector={"points": ids_to_delete})
#     print(f"Deleted {len(ids_to_delete)} Q/A-polluted entries from database.")
# else:
#     print("No Q/A-polluted records detected.")

In [10]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
adapter_path = "/kaggle/working/ragbot_databse_new/lora_adapter_latest"

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=token,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [11]:
# Load model & tokenizer
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

USE_MERGED = False  # Set True to try merge
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16, token=token, trust_remote_code=True)

# Load and merge the adapter
model = PeftModel.from_pretrained(model, adapter_path, trust_remote_code=True)
# model = model.merge_and_unload()  # Merge adapter weights into base model
# Try either lazy loading OR merging
# USE_MERGED = False  # Set True to try merge

if USE_MERGED:
    model = model.merge_and_unload()
    print("LoRA adapter merged into base model.")
else:
    print("LoRA adapter loaded (lazy mode).")

model.eval()

# Create text generation pipeline
# text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Confirm model type
print("🔍 Model type:", type(model))
print("🎯 Model in eval mode?", model.training == False)

# Print PEFT adapter info
if hasattr(model, "peft_config"):
    print("🧾 PEFT Config loaded:")
    print(model.peft_config)
else:
    print("⚠️ Warning: No PEFT config found. Adapter may not have loaded correctly.")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LoRA adapter loaded (lazy mode).
🔍 Model type: <class 'peft.peft_model.PeftModelForCausalLM'>
🎯 Model in eval mode? True
🧾 PEFT Config loaded:
{'default': LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='meta-llama/Meta-Llama-3-8B-Instruct', revision=None, inference_mode=True, r=8, target_modules={'v_proj', 'q_proj'}, exclude_modules=None, lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None)}


In [12]:
print(type(model))


<class 'peft.peft_model.PeftModelForCausalLM'>


In [13]:
!nvidia-smi


Tue Aug  5 15:38:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   67C    P0             29W /   70W |    9469MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# import torch
# torch.cuda.empty_cache()


In [14]:
test_prompt = "[INST] Summarize: Hypertension leads to kidney damage. [/INST]"

# Tokenize and move to correct device
inputs = tokenizer(test_prompt, return_tensors="pt")
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# Generate response
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=50)

# Decode and print result
print("🧠 Model Output:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


🧠 Model Output:
 [INST] Summarize: Hypertension leads to kidney damage. [/INST] Hypertension (high blood pressure) is a major risk factor for kidney damage. High blood pressure damages the blood vessels in the kidneys, causing them to leak protein into the urine. This can lead to chronic kidney disease (CKD), which can


In [23]:
query = """
Patient Report:
- Age: 62
- Symptoms: Persistent headaches, dizziness, occasional vision issues
- Blood Pressure: 165/105 mmHg
- Serum Creatinine: 1.6 mg/dL
- Urinalysis: Presence of protein
- History: Long-standing untreated high blood pressure

Please analyze and highlight any significant findings or possible diagnoses.
"""

query_vector = embedding_model.encode(query)

search_result = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3,
    with_payload=True
)
for i, hit in enumerate(search_result):
    print(f"Result {i+1}:")
    print("Payload:", hit.payload)
    print("Score:", hit.score)
    print("-" * 40)

# retrieved_chunks = [hit.payload.get("chunk", "") for hit in search_result]
# context = "\n".join(retrieved_chunks)

# final_prompt = f"""[INST]
# Context:
# {context}

# User Query:
# {query}
# [/INST]"""

# inputs = tokenizer(final_prompt, return_tensors="pt").to(model.device)
# with torch.no_grad():
#     output = model.generate(**inputs, max_new_tokens=200)

# print(tokenizer.decode(output[0], skip_special_tokens=True))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Result 1:
Payload: {'chunk_text': 'vates clotting mechanisms leading to a microangiopathic hemolytic anemia thus perpetuating the vascular pathology and 2 activation of the contributes to the acceleration and maintenance of blood pressure elevation and in turn to vascular injury malignant hypertension is most likely to develop in a previously hypertensive individual usually in the third or fourth decade of life there is a higher incidence among men particularly black men the presenting symptoms are usually neurologic dizziness headache blurring of vision altered states of consciousness and focal or generalized seizures cardiac decompensation and renal failure appear thereafter renal abnormalities include a rapid rise in serum creatinine hematuria at times macroscopic proteinuria and red and white blood cell casts in the sediment nephrotic syndrome may be present elevated plasma aldosterone levels cause hypokalemic metabolic alkalosis in the early phase uremic acidosis and hyperkalemia 

/tmp/ipykernel_36/3898579300.py:15: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qdrant.search(


In [34]:
retrieved_chunks = [hit.payload.get("chunk_text") for hit in search_result]
context = "\n".join(retrieved_chunks)

final_prompt = f"""
[INST] <<SYS>>
You are a helpful and knowledgeable medical assistant. Use the given medical context to answer the user's question accurately and precisely.
<</SYS>>

Context:
{context}

User Query:
{query}

Answer in detail based on the above context:
[/INST]
"""

input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    output_ids = model.generate(
        **input_ids,
        max_new_tokens=512,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

# Slice only the generated tokens (excluding the prompt)
generated_tokens = output_ids[0][input_ids["input_ids"].shape[-1]:]
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(generated_text)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


You are a helpful and knowledgeable medical assistant. Use the given medical context to answer the user's question accurately and precisely.
<</SYS>>



Based on the patient's report, I would highlight the following significant findings:

* Persistent headaches, dizziness, and occasional vision issues: These symptoms could be indicative of hypertension-related complications, such as hypertensive encephalopathy, hypertensive retinopathy, or hypertensive nephrosclerosis.
* Blood Pressure: 165/105 mmHg: This is a high blood pressure reading, which is consistent with hypertension.
* Serum Creatinine: 1.6 mg/dL: This is an elevated serum creatinine level, which could be indicative of kidney damage or dysfunction.
* Urinalysis: Presence of protein: This could be indicative of proteinuria, which is a common finding in patients with hypertension-related kidney disease.

Given the patient's history of long-standing untreated high blood pressure, I would suspect that the patient has hypertensive

In [35]:
print("------Prompt Preview------")
print(prompt[:500])  # or just print(prompt)

print("------Generated Answer------")
print(generated_text)


------Prompt Preview------

[INST] <<SYS>>
You are a helpful and knowledgeable medical assistant. Use the given medical context to answer the user's question accurately and precisely.
<</SYS>>

Context:
vates clotting mechanisms leading to a microangiopathic hemolytic anemia thus perpetuating the vascular pathology and 2 activation of the contributes to the acceleration and maintenance of blood pressure elevation and in turn to vascular injury malignant hypertension is most likely to develop in a previously hypertensive i
------Generated Answer------
You are a helpful and knowledgeable medical assistant. Use the given medical context to answer the user's question accurately and precisely.
<</SYS>>



Based on the patient's report, I would highlight the following significant findings:

* Persistent headaches, dizziness, and occasional vision issues: These symptoms could be indicative of hypertension-related complications, such as hypertensive encephalopathy, hypertensive retinopathy, or

In [ ]:
# # 1. Check special tokens
# print("[INST] token id:", tokenizer.convert_tokens_to_ids("[INST]"))
# print("[/INST] token id:", tokenizer.convert_tokens_to_ids("[/INST]"))

# # 2. Add special tokens if needed
# special_tokens_dict = {'additional_special_tokens': ['[INST]', '[/INST]']}
# num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)

# # 3. Set pad_token if not set
# tokenizer.pad_token = tokenizer.eos_token

# # 4. Resize model embeddings ONLY if new tokens were added
# if num_new_tokens > 0:
#     model.resize_token_embeddings(len(tokenizer))


In [ ]:
# # Define build_prompt with bug fix and sharper instructions
# def build_prompt(user_query, retrieved_chunks, max_context_tokens=1700):
#     unique_chunks = list(dict.fromkeys(retrieved_chunks))
#     final_chunks = []
#     total_tokens = 0
#     for chunk in unique_chunks:
#         chunk_tokens = len(tokenizer.encode(chunk))
#         if total_tokens + chunk_tokens > max_context_tokens:
#             break
#         if chunk.strip():
#             final_chunks.append(chunk.strip())
#             total_tokens += chunk_tokens
#     joined_chunks = "\n\n".join(final_chunks)

#     instruction = f"""
# You are a medical assistant. Given the patient report and ONLY the RELEVANT MEDICAL KNOWLEDGE provided, write a concise clinical summary and possible diagnosis.
# - Do NOT copy directly from the knowledge.
# - Synthesize in your own words and be as brief as possible.
# - Use only information from the knowledge context.

# Patient Report:
# {user_query.strip()}

# Relevant Medical Knowledge:
# {joined_chunks}

# Your Answer:
# """
#     return f"<s>[INST] {instruction.strip()} [/INST]"


In [ ]:
# # 🧪 Test your prompt construction and tokenization-decoding loop

# # Example input (replace with real test case)
# user_query = "What are the symptoms of diabetes?"
# retrieved_chunks = [
#     "Diabetes symptoms include increased thirst, frequent urination, hunger, fatigue, and blurred vision.",
#     "In some cases, symptoms may be mild or absent, especially in type 2 diabetes during early stages."
# ]

# # ✅ Build the full prompt for LLaMA-style models
# def build_prompt(user_query, retrieved_chunks):
#     context = "\n".join(retrieved_chunks)
#     return f"<s>[INST] You are a helpful medical assistant.\nContext:\n{context}\n\nQuestion: {user_query} [/INST]"

# # 1. Build prompt
# prompt = build_prompt(user_query, retrieved_chunks)
# print("📌 Full Prompt:\n", prompt)

# # 2. Tokenize
# inputs = tokenizer(prompt, return_tensors="pt")
# input_ids = inputs["input_ids"]

# # 3. Decode back
# decoded = tokenizer.decode(input_ids[0], skip_special_tokens=False)
# print("\n🧾 Decoded Prompt:\n", decoded)

# # 4. Print prompt length in tokens
# print(f"\n🧮 Prompt token length: {input_ids.shape[1]}")


In [ ]:
# # vector search + generate response

# def generate_response(user_query, top_k=5):
#     # Step 1: Embed user query
#     query_embedding = embedding_model.encode([user_query])[0].tolist()

#     # Step 2: Vector search in Qdrant
#     search_result = qdrant.search(
#         collection_name=collection_name,
#         query_vector=query_embedding,
#         limit=top_k,
#         with_payload=True
#     )

#     # Step 3: Collect retrieved documents
#     retrieved_chunks = []
#     for hit in search_result:
#         payload = None
#         if hasattr(hit, "payload"):
#             payload = hit.payload
#         elif isinstance(hit, list) and hasattr(hit[0], "payload"):
#             payload = hit[0].payload
#         if payload:
#             val = payload.get("chunk_text") or payload.get("chunk") or payload.get("content") or payload.get("text") 
#             if val: retrieved_chunks.append(val)

#     # print("Retrieved Chunks:")
#     # for i, chunk in enumerate(retrieved_chunks): print(f"Chunk {i}:", repr(chunk))

#     if not retrieved_chunks:
#         return "I don't know.", None, None


#     # Step 4: Create prompt from context + query
#     prompt = build_prompt(user_query, retrieved_chunks)

#     # Step 5: Generate response using merged model
#     inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
#     outputs = model.generate(**inputs, max_new_tokens=512, do_sample=True, temperature=0.7)
    
#     generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

#     # For LLaMA-style models, remove everything before [/INST]
#     if "[/INST]" in generated_text:
#         response_text = generated_text.split("[/INST]")[-1].strip()
#     else:
#         response_text = generated_text.strip()

#     # # Step 6: Save query+response to vector DB
#     # combined_text = f"Prompt: {user_query}\nAnswer: {response_text}"
#     # combined_embedding = embedding_model.encode([combined_text])[0].tolist()

#     # qdrant.upsert(
#     #     collection_name=collection_name,
#     #     points=[
#     #         PointStruct(id=str(uuid.uuid4()), vector=combined_embedding, payload={"text": combined_text})
#     #     ]
#     # )

#     # Step 7: Return response to user
#     # print("Effective Prompt:\n", prompt)
    
#     # print("Generated Raw Output:\n", generated_text)
#     # print("Extracted Response:\n", response_text)
    
#     return response_text, prompt, generated_text

#     # print("="*20)
#     # print("Prompt Sent:\n", prompt)
#     # print("="*20)
#     # print("Raw Output:\n", generated_text)
#     # print("="*20)
#     # print("Final Answer:\n", response_text)
#     # print("="*20)




    

    


In [ ]:
# user_query = """
# Patient Report:
# - Age: 62
# - Symptoms: Persistent headaches, dizziness, occasional vision issues
# - Blood Pressure: 165/105 mmHg
# - Serum Creatinine: 1.6 mg/dL
# - Urinalysis: Presence of protein
# - History: Long-standing untreated high blood pressure
# """

# output = generate_response(user_query)

# if isinstance(output, tuple):
#     response, prompt_used, raw_output = output
# else:
#     response, prompt_used, raw_output = output, None, None

# print("----- Final Answer -----")
# print(response)

# if prompt_used:
#     print("\n----- Prompt Sent to Model -----")
#     print(prompt_used)


In [ ]:
# # View existing documents in Qdrant
# search = qdrant.scroll(
#     collection_name=collection_name,
#     limit=5,
#     with_payload=True
# )

# for point in search[0]:
#     print("ID:", point.id)
#     print("Payload:", point.payload)
#     print("---")


In [ ]:
print(qdrant.count(collection_name=collection_name).count)


In [ ]:
test_fact = "Hypertensive nephropathy is kidney damage caused by long-standing high blood pressure. Clinical features include proteinuria and elevated creatinine."
test_emb = embedding_model.encode([test_fact])[0].tolist()
qdrant.upsert(
    collection_name=collection_name,
    points=[PointStruct(id=str(uuid.uuid4()), vector=test_emb, payload={"chunk_text": test_fact})]
)
print("Ingested test fact.")

In [ ]:
query = """
Patient Report:
- Age: 62
- Symptoms: Persistent headaches, dizziness, occasional vision issues
- Blood Pressure: 165/105 mmHg
- Serum Creatinine: 1.6 mg/dL
- Urinalysis: Presence of protein
- History: Long-standing untreated high blood pressure

Please analyze and highlight any significant findings or possible diagnoses.
"""
# ansswer = generate_response(query)
# print("Generated Answer:\n", answer)


response, prompt, raw = generate_response(query)
# print("\nPrompt sent to LLM:\n", prompt)
# print("\nModel output:\n", raw)
# print("\nFINAL clinical summary:\n", response)
print("----- Final Answer -----")
print(response)

print("\n----- Prompt Sent to Model -----")
print(prompt_used)

In [ ]:
# # function definition ends here

# resp, prompt, gen_out = generate_response(query)
# print("Prompt Sent to Model:\n", prompt)
# print("\nModel Output:\n", gen_out)
# print("\nFinal Response:\n", resp)
